# 🎬 MyNextMovie Interactive GUI

Interactive GUI interface for the movie recommendation system using ipywidgets.
This notebook provides a user-friendly interface to test all three recommendation algorithms.

In [1]:
# Import required libraries
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import warnings
warnings.filterwarnings('ignore')

# Add src directory to path
sys.path.append('../src')

# Import our recommendation system
from data_loader import DataLoader
from recommendation_engine import (
    PopularityRecommender,
    ContentBasedRecommender,
    CollaborativeFilteringRecommender,
    HybridRecommender
)

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [2]:
# Initialize the recommendation system
print("🔄 Initializing MyNextMovie Recommendation System...")

# Load data
data_loader = DataLoader('../data/raw')
movies_df, ratings_df = data_loader.load_data()
movies_with_stats = data_loader.preprocess_data()

# Initialize recommenders
popularity_rec = PopularityRecommender(movies_with_stats)
content_rec = ContentBasedRecommender(movies_df)
collaborative_rec = CollaborativeFilteringRecommender(ratings_df, movies_df)
hybrid_rec = HybridRecommender(popularity_rec, content_rec, collaborative_rec)

print(f"🎉 System initialized successfully!")
print(f"📊 Loaded {len(movies_df)} movies and {len(ratings_df)} ratings")

🔄 Initializing MyNextMovie Recommendation System...
Movies loaded: 10329 records
Ratings loaded: 105339 records
Data preprocessing completed successfully!
🎉 System initialized successfully!
📊 Loaded 10329 movies and 105339 ratings


In [3]:
# Helper functions for the GUI

def get_unique_genres():
    """Get list of unique genres for dropdown"""
    all_genres = set()
    for genres_str in movies_df['genres'].dropna():
        genres = genres_str.split('|')
        all_genres.update(genres)
    return sorted(list(all_genres))

def get_sample_movies(n=20):
    """Get sample movie titles for dropdown"""
    popular_movies = movies_with_stats.nlargest(n, 'num_ratings')['title'].tolist()
    return sorted(popular_movies)

def get_sample_users(n=20):
    """Get sample user IDs"""
    user_counts = ratings_df['userId'].value_counts().head(n)
    return sorted(user_counts.index.tolist())

def style_dataframe(df):
    """Apply styling to dataframe for better display"""
    return df.style.set_properties(**{
        'background-color': '#f8f9fa',
        'color': '#333',
        'border': '1px solid #dee2e6',
        'text-align': 'left'
    }).set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#007bff'), ('color', 'white'), ('font-weight', 'bold')]}
    ])

def show_loading(output_widget, message="Processing..."):
    """Show loading spinner with message"""
    with output_widget:
        clear_output()
        loading_html = f"""
        <div style="text-align: center; padding: 20px;">
            <div style="display: inline-block; animation: spin 1s linear infinite; font-size: 20px;">⏳</div>
            <p style="color: #007bff; font-weight: bold; margin-top: 10px;">{message}</p>
        </div>
        <style>
        @keyframes spin {{
            0% {{ transform: rotate(0deg); }}
            100% {{ transform: rotate(360deg); }}
        }}
        </style>
        """
        display(HTML(loading_html))

def show_progress_bar(output_widget, progress=0, total=100, message="Processing..."):
    """Show progress bar with percentage"""
    with output_widget:
        clear_output()
        percentage = (progress / total) * 100
        progress_html = f"""
        <div style="text-align: center; padding: 20px;">
            <p style="color: #007bff; font-weight: bold; margin-bottom: 10px;">{message}</p>
            <div style="width: 100%; background-color: #e9ecef; border-radius: 10px; padding: 3px;">
                <div style="width: {percentage}%; height: 20px; background-color: #007bff; border-radius: 7px; transition: width 0.3s ease;"></div>
            </div>
            <p style="color: #6c757d; margin-top: 5px;">{percentage:.1f}% Complete</p>
        </div>
        """
        display(HTML(progress_html))

def clear_all_widgets():
    """Comprehensive widget cleanup - remove all existing widget instances"""
    # List of all widget variable names that might exist
    widget_vars = [
        'popularity_box', 'popularity_button', 'popularity_output', 'genre_dropdown', 
        'min_reviews_slider', 'num_recs_slider',
        'content_box', 'content_button', 'content_output', 'movie_text', 'content_num_recs',
        'collab_box', 'collab_button', 'collab_output', 'user_text', 'collab_num_recs', 'k_users_slider',
        'hybrid_box', 'hybrid_button', 'hybrid_output', 'hybrid_user_id', 'hybrid_num_recs', 
        'collab_weight', 'popularity_weight',
        'analytics_box', 'analytics_button', 'analytics_output',
        'demo_box', 'demo_button', 'demo_output'
    ]
    
    # Close any existing widgets
    for var_name in widget_vars:
        if var_name in globals():
            try:
                widget = globals()[var_name]
                if hasattr(widget, 'close'):
                    widget.close()
                # Remove from global namespace
                del globals()[var_name]
            except:
                pass
    
    # Force clear the current cell output
    clear_output(wait=True)

# Get data for dropdowns
unique_genres = get_unique_genres()
sample_movies = get_sample_movies()
sample_users = get_sample_users()

print(f"📋 Found {len(unique_genres)} genres, {len(sample_movies)} popular movies, {len(sample_users)} active users")
print("🧹 Widget cleanup functions loaded")

📋 Found 20 genres, 20 popular movies, 20 active users
🧹 Widget cleanup functions loaded


## 🔥 Popularity-Based Recommender GUI

In [ ]:
# Create widgets for popularity-based recommendations
# Comprehensive cleanup before creating new widgets
clear_all_widgets()

# Create fresh widget instances
genre_dropdown = widgets.Dropdown(
    options=unique_genres,
    value='Action',
    description='Genre:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

min_reviews_slider = widgets.IntSlider(
    value=50,
    min=1,
    max=200,
    step=10,
    description='Min Reviews:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

num_recs_slider = widgets.IntSlider(
    value=10,
    min=1,
    max=20,
    step=1,
    description='Number of Recommendations:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

popularity_button = widgets.Button(
    description='Get Popular Movies',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

popularity_output = widgets.Output()

def style_dataframe_with_separate_columns(df):
    """Apply styling to dataframe and separate movie name from year"""
    styled_df = df.copy()
    
    # If there's a Movie Title column, separate it into Movie Name and Year
    if 'Movie Title' in styled_df.columns:
        # Extract year and clean movie name
        styled_df['Year'] = styled_df['Movie Title'].str.extract(r'\((\d{4})\)$').fillna('')
        styled_df['Movie Name'] = styled_df['Movie Title'].str.replace(r'\s*\(\d{4}\)$', '', regex=True)
        # Reorder columns
        cols = ['S.No.'] if 'S.No.' in styled_df.columns else []
        cols.extend(['Movie Name', 'Year'])
        if 'Average Movie Rating' in styled_df.columns:
            cols.append('Average Movie Rating')
        if 'Num Reviews' in styled_df.columns:
            cols.append('Num Reviews')
        # Add any remaining columns
        for col in styled_df.columns:
            if col not in cols and col != 'Movie Title':
                cols.append(col)
        styled_df = styled_df[cols]
        # Drop original Movie Title column
        if 'Movie Title' in styled_df.columns:
            styled_df = styled_df.drop('Movie Title', axis=1)
    
    return styled_df.style.set_properties(**{
        'background-color': '#f8f9fa',
        'color': '#333',
        'border': '1px solid #dee2e6',
        'text-align': 'left'
    }).set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#007bff'), ('color', 'white'), ('font-weight', 'bold')]}
    ])

def on_popularity_button_click(b):
    show_loading(popularity_output, "🔄 Getting popular movies...")
    
    try:
        # Simulate processing steps with progress updates
        show_progress_bar(popularity_output, 25, 100, "Loading movie data...")
        
        recommendations = popularity_rec.recommend(
            genre=genre_dropdown.value,
            min_reviews_threshold=min_reviews_slider.value,
            num_recommendations=num_recs_slider.value
        )
        
        show_progress_bar(popularity_output, 75, 100, "Generating recommendations...")
        
        with popularity_output:
            clear_output()
            
            if recommendations.empty:
                print("❌ No movies found matching the criteria.")
            else:
                print(f"🎬 Top {len(recommendations)} Popular {genre_dropdown.value} Movies:")
                display(style_dataframe_with_separate_columns(recommendations))
                
                # Create a single bar chart - consolidate all plotting into one block
                fig, ax = plt.subplots(figsize=(12, 6))
                # Extract clean movie names for the chart
                movie_names = recommendations['Movie Title'].str.replace(r'\s*\(\d{4}\)$', '', regex=True)
                ax.barh(movie_names[::-1], recommendations['Average Movie Rating'][::-1], color='skyblue')
                ax.set_xlabel('Average Rating')
                ax.set_title(f'Top {genre_dropdown.value} Movies by Rating')
                plt.tight_layout()
                plt.show()
                
    except Exception as e:
        with popularity_output:
            clear_output()
            print(f"❌ Error: {str(e)}")

popularity_button.on_click(on_popularity_button_click)

# Display popularity recommender interface
popularity_box = widgets.VBox([
    widgets.HTML("<h3>🔥 Popularity-Based Recommendations</h3>"),
    widgets.HBox([genre_dropdown]),
    widgets.HBox([min_reviews_slider]),
    widgets.HBox([num_recs_slider]),
    widgets.HBox([popularity_button]),
    popularity_output
], layout=widgets.Layout(border='2px solid #007bff', padding='10px', margin='10px'))

display(popularity_box)

## 🎯 Content-Based Recommender GUI

In [5]:
# Create widgets for content-based recommendations
# Clear any existing widgets first
clear_output(wait=True)

movie_text = widgets.Text(
    value='',
    placeholder='Type movie title here (e.g., Toy Story, The Matrix, Titanic)...',
    description='Movie Title:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

content_num_recs = widgets.IntSlider(
    value=10,
    min=1,
    max=20,
    step=1,
    description='Number of Recommendations:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

content_button = widgets.Button(
    description='Find Similar Movies',
    button_style='info',
    layout=widgets.Layout(width='200px', height='40px')
)

content_output = widgets.Output()

def on_content_button_click(b):
    # Get movie title from text input
    movie_title = movie_text.value.strip()
    
    # Clean the movie title (remove year if present)
    if '(' in movie_title and ')' in movie_title:
        movie_title = movie_title.split('(')[0].strip()
    
    if not movie_title:
        with content_output:
            clear_output()
            print("❌ Please enter a movie title.")
        return
    
    show_loading(content_output, f"🔄 Finding movies similar to '{movie_title}'...")
    
    try:
        show_progress_bar(content_output, 30, 100, "Analyzing movie features...")
        
        recommendations = content_rec.recommend(
            movie_title=movie_title,
            num_recommendations=content_num_recs.value
        )
        
        show_progress_bar(content_output, 80, 100, "Computing similarity scores...")
        
        with content_output:
            clear_output()
            
            if recommendations.empty:
                print("❌ No similar movies found or movie not in database.")
                print("💡 Try these suggestions:")
                print("   - Check the spelling of the movie title")
                print("   - Try a more popular movie like 'Toy Story', 'The Matrix', or 'Titanic'")
                print("   - Use the exact title as it appears in the database")
                print("   - Enter just the movie name without the year")
            else:
                print(f"🎯 Movies Similar to '{movie_title}':")
                display(style_dataframe_with_separate_columns(recommendations))
                
                # Show genre distribution - consolidate all plotting into one block
                similar_movie_ids = []
                for title in recommendations['Movie Title']:
                    # Clean the title for searching
                    clean_title = str(title)
                    if '(' in clean_title and ')' in clean_title:
                        clean_title = clean_title.split('(')[0].strip()
                    
                    movie_match = movies_df[movies_df['title'].str.contains(clean_title, case=False, na=False)]
                    if not movie_match.empty:
                        similar_movie_ids.append(movie_match.iloc[0]['movieId'])
                
                if similar_movie_ids:
                    similar_movies_data = movies_df[movies_df['movieId'].isin(similar_movie_ids)]
                    all_genres = []
                    for genres_str in similar_movies_data['genres'].dropna():
                        all_genres.extend(genres_str.split('|'))
                    
                    if all_genres:
                        genre_counts = pd.Series(all_genres).value_counts().head(8)
                        
                        fig, ax = plt.subplots(figsize=(10, 6))
                        ax.pie(genre_counts.values, labels=genre_counts.index, autopct='%1.1f%%', startangle=90)
                        ax.set_title(f'Genre Distribution of Movies Similar to "{movie_title}"')
                        ax.axis('equal')
                        plt.tight_layout()
                        plt.show()
                
    except Exception as e:
        with content_output:
            clear_output()
            print(f"❌ Error: {str(e)}")

content_button.on_click(on_content_button_click)

# Display content-based recommender interface
content_box = widgets.VBox([
    widgets.HTML("<h3>🎯 Content-Based Recommendations</h3>"),
    widgets.HTML("<p>Enter a movie title to find similar movies based on genres, cast, and other features.</p>"),
    widgets.HTML("<p><strong>Note:</strong> Enter just the movie name without the year for better search results.</p>"),
    widgets.HBox([movie_text]),
    widgets.HBox([content_num_recs]),
    widgets.HBox([content_button]),
    content_output
], layout=widgets.Layout(border='2px solid #17a2b8', padding='10px', margin='10px'))

display(content_box)

## 👥 Collaborative Filtering Recommender GUI

In [6]:
# Create widgets for collaborative filtering recommendations
# Clear any existing widgets first
clear_output(wait=True)

user_text = widgets.IntText(
    value=1,
    description='Enter User ID:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

collab_num_recs = widgets.IntSlider(
    value=10,
    min=1,
    max=20,
    step=1,
    description='Number of Recommendations:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

k_users_slider = widgets.IntSlider(
    value=100,
    min=10,
    max=300,
    step=10,
    description='Similar Users (K):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

collab_button = widgets.Button(
    description='Get Personal Recommendations',
    button_style='warning',
    layout=widgets.Layout(width='250px', height='40px')
)

collab_output = widgets.Output()

def on_collab_button_click(b):
    # Use text input
    user_id = user_text.value
    
    show_loading(collab_output, f"🔄 Getting personalized recommendations for User {user_id}...")
    
    try:
        show_progress_bar(collab_output, 20, 100, "Loading user rating history...")
        
        show_progress_bar(collab_output, 50, 100, "Finding similar users...")
        
        recommendations = collaborative_rec.recommend(
            user_id=user_id,
            num_recommendations=collab_num_recs.value,
            k_similar_users=k_users_slider.value
        )
        
        show_progress_bar(collab_output, 90, 100, "Generating recommendations...")
        
        with collab_output:
            clear_output()
            
            if recommendations.empty:
                print("❌ No recommendations found for this user.")
            else:
                print(f"👥 Personalized Recommendations for User {user_id}:")
                display(style_dataframe_with_separate_columns(recommendations))
                
                # Show user's rating history with separate movie name and year
                user_ratings = ratings_df[ratings_df['userId'] == user_id]
                if not user_ratings.empty:
                    user_ratings_with_titles = user_ratings.merge(movies_df[['movieId', 'title']], on='movieId')
                    top_rated = user_ratings_with_titles.nlargest(10, 'rating')[['title', 'rating']]
                    
                    # Separate movie name and year for display
                    top_rated_display = top_rated.copy()
                    top_rated_display['Movie Name'] = top_rated_display['title'].str.replace(r'\s*\(\d{4}\)$', '', regex=True)
                    top_rated_display['Year'] = top_rated_display['title'].str.extract(r'\((\d{4})\)$').fillna('')
                    top_rated_display = top_rated_display[['Movie Name', 'Year', 'rating']].reset_index(drop=True)
                    top_rated_display.columns = ['Movie Name', 'Year', 'Rating']
                    
                    print(f"\n📊 User {user_id}'s Top Rated Movies:")
                    display(style_dataframe(top_rated_display))
                    
                    # Create a single combined plot instead of separate subplots
                    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
                    
                    # Rating distribution histogram
                    ax1.hist(user_ratings['rating'], bins=10, alpha=0.7, color='orange')
                    ax1.set_title(f'User {user_id} Rating Distribution')
                    ax1.set_xlabel('Rating')
                    ax1.set_ylabel('Frequency')
                    
                    # Rating counts bar chart
                    rating_counts = user_ratings['rating'].value_counts().sort_index()
                    ax2.bar(rating_counts.index, rating_counts.values, alpha=0.7, color='green')
                    ax2.set_title(f'User {user_id} Ratings by Score')
                    ax2.set_xlabel('Rating Score')
                    ax2.set_ylabel('Count')
                    
                    plt.tight_layout()
                    plt.show()
                
    except Exception as e:
        with collab_output:
            clear_output()
            print(f"❌ Error: {str(e)}")

collab_button.on_click(on_collab_button_click)

# Display collaborative filtering interface
collab_box = widgets.VBox([
    widgets.HTML("<h3>👥 Collaborative Filtering Recommendations</h3>"),
    widgets.HBox([user_text]),
    widgets.HBox([collab_num_recs]),
    widgets.HBox([k_users_slider]),
    widgets.HBox([collab_button]),
    collab_output
], layout=widgets.Layout(border='2px solid #ffc107', padding='10px', margin='10px'))

display(collab_box)

## 🌟 Hybrid Recommender GUI

In [ ]:
# Create widgets for hybrid recommendations
# Clear any existing widgets first
clear_output(wait=True)

hybrid_user_id = widgets.IntText(
    value=1,
    description='User ID:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

hybrid_num_recs = widgets.IntSlider(
    value=10,
    min=1,
    max=20,
    step=1,
    description='Number of Recommendations:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

collab_weight = widgets.FloatSlider(
    value=0.6,
    min=0.0,
    max=1.0,
    step=0.1,
    description='Collaborative Weight:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

popularity_weight = widgets.FloatSlider(
    value=0.4,
    min=0.0,
    max=1.0,
    step=0.1,
    description='Popularity Weight:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

hybrid_button = widgets.Button(
    description='Get Hybrid Recommendations',
    button_style='danger',
    layout=widgets.Layout(width='250px', height='40px')
)

hybrid_output = widgets.Output()

def update_weights(*args):
    """Auto-update weights to sum to 1"""
    if collab_weight.value + popularity_weight.value != 1.0:
        popularity_weight.value = 1.0 - collab_weight.value

collab_weight.observe(update_weights, 'value')

def on_hybrid_button_click(b):
    user_id = hybrid_user_id.value
    weights = {
        'collaborative': collab_weight.value,
        'popularity': popularity_weight.value
    }
    
    show_loading(hybrid_output, f"🔄 Computing hybrid recommendations for User {user_id}...")
    
    try:
        show_progress_bar(hybrid_output, 25, 100, "Initializing collaborative filtering...")
        
        show_progress_bar(hybrid_output, 50, 100, "Computing popularity scores...")
        
        show_progress_bar(hybrid_output, 75, 100, "Combining algorithms...")
        
        recommendations = hybrid_rec.get_combined_recommendations(
            user_id=user_id,
            num_recommendations=hybrid_num_recs.value,
            weights=weights
        )
        
        with hybrid_output:
            clear_output()
            print(f"📊 Weights: Collaborative={weights['collaborative']:.1f}, Popularity={weights['popularity']:.1f}")
            
            if recommendations.empty:
                print("❌ No hybrid recommendations found.")
            else:
                print(f"🌟 Hybrid Recommendations for User {user_id}:")
                display(style_dataframe_with_separate_columns(recommendations))
                
                # Compare with individual methods
                collab_recs = collaborative_rec.recommend(user_id, 5)
                pop_recs = popularity_rec.recommend('Drama', 20, 5)
                
                # Clean up the movie titles for comparison
                def clean_movie_titles(df):
                    if 'Movie Title' in df.columns:
                        df_clean = df.copy()
                        df_clean['Movie Title'] = df_clean['Movie Title'].str.replace(r'\s*\(\d{4}\)$', '', regex=True)
                        return df_clean['Movie Title'].head(5).tolist()
                    return [''] * 5
                
                hybrid_titles = clean_movie_titles(recommendations)
                collab_titles = clean_movie_titles(collab_recs)
                pop_titles = clean_movie_titles(pop_recs)
                
                comparison_data = {
                    'Rank': list(range(1, 6)),
                    'Hybrid': hybrid_titles + [''] * (5 - len(hybrid_titles)),
                    'Collaborative': collab_titles + [''] * (5 - len(collab_titles)),
                    'Popularity': pop_titles + [''] * (5 - len(pop_titles))
                }
                
                comparison_df = pd.DataFrame(comparison_data)
                print("\n📈 Method Comparison (Top 5 Movie Names):")
                display(style_dataframe(comparison_df))
                
    except Exception as e:
        with hybrid_output:
            clear_output()
            print(f"❌ Error: {str(e)}")

hybrid_button.on_click(on_hybrid_button_click)

# Display hybrid recommender interface
hybrid_box = widgets.VBox([
    widgets.HTML("<h3>🌟 Hybrid Recommendations</h3>"),
    widgets.HBox([hybrid_user_id]),
    widgets.HBox([hybrid_num_recs]),
    widgets.HTML("<h4>Algorithm Weights:</h4>"),
    widgets.HBox([collab_weight]),
    widgets.HBox([popularity_weight]),
    widgets.HBox([hybrid_button]),
    hybrid_output
], layout=widgets.Layout(border='2px solid #dc3545', padding='10px', margin='10px'))

display(hybrid_box)

## 📊 Data Analytics Dashboard

In [ ]:
# Create analytics dashboard
# Clear any existing widgets first
clear_output(wait=True)

analytics_button = widgets.Button(
    description='Show Data Analytics',
    button_style='primary',
    layout=widgets.Layout(width='200px', height='40px')
)

analytics_output = widgets.Output()

def on_analytics_button_click(b):
    show_loading(analytics_output, "📊 Generating analytics dashboard...")
    
    try:
        show_progress_bar(analytics_output, 20, 100, "Computing basic statistics...")
        
        # Basic statistics
        try:
            genre_counts = movies_df['genres'].str.split('|').explode().value_counts()
            most_popular_genre = genre_counts.index[0] if len(genre_counts) > 0 else 'Unknown'
        except:
            most_popular_genre = 'Unknown'
            
        stats = {
            'Total Movies': len(movies_df),
            'Total Users': ratings_df['userId'].nunique(),
            'Total Ratings': len(ratings_df),
            'Average Rating': round(ratings_df['rating'].mean(), 2),
            'Unique Genres': len(get_unique_genres()),
            'Most Popular Genre': most_popular_genre,
            'Data Sparsity': round(((len(movies_df) * ratings_df['userId'].nunique() - len(ratings_df)) / (len(movies_df) * ratings_df['userId'].nunique()) * 100), 2)
        }
        
        show_progress_bar(analytics_output, 60, 100, "Creating visualizations...")
        
        with analytics_output:
            clear_output()
            
            print("📊 MyNextMovie Data Analytics Dashboard")
            print("=" * 50)
            
            stats_df = pd.DataFrame(list(stats.items()), columns=['Metric', 'Value'])
            display(style_dataframe(stats_df))
            
            # Create visualizations
            fig, axes = plt.subplots(2, 2, figsize=(15, 12))
            
            # Rating distribution
            try:
                axes[0, 0].hist(ratings_df['rating'], bins=10, alpha=0.7, color='skyblue')
                axes[0, 0].set_title('Rating Distribution')
                axes[0, 0].set_xlabel('Rating')
                axes[0, 0].set_ylabel('Frequency')
            except Exception as e:
                axes[0, 0].text(0.5, 0.5, f'Error: {str(e)}', ha='center', va='center')
                axes[0, 0].set_title('Rating Distribution (Error)')
            
            # Top genres
            try:
                all_genres = movies_df['genres'].str.split('|').explode().value_counts().head(10)
                if len(all_genres) > 0:
                    axes[0, 1].barh(all_genres.index[::-1], all_genres.values[::-1], color='lightcoral')
                    axes[0, 1].set_title('Top 10 Genres')
                    axes[0, 1].set_xlabel('Number of Movies')
                else:
                    axes[0, 1].text(0.5, 0.5, 'No genre data available', ha='center', va='center')
                    axes[0, 1].set_title('Top 10 Genres (No Data)')
            except Exception as e:
                axes[0, 1].text(0.5, 0.5, f'Error: {str(e)}', ha='center', va='center')
                axes[0, 1].set_title('Top 10 Genres (Error)')
            
            # Movies per year
            try:
                # Create a copy to avoid modifying original dataframe
                movies_temp = movies_df.copy()
                movies_temp['year'] = movies_temp['title'].str.extract(r'\((\d{4})\)').astype(float)
                year_counts = movies_temp['year'].dropna().value_counts().sort_index().tail(20)
                
                if len(year_counts) > 0:
                    axes[1, 0].plot(year_counts.index, year_counts.values, marker='o', color='green')
                    axes[1, 0].set_title('Movies Released by Year (Last 20 Years)')
                    axes[1, 0].set_xlabel('Year')
                    axes[1, 0].set_ylabel('Number of Movies')
                    axes[1, 0].tick_params(axis='x', rotation=45)
                else:
                    axes[1, 0].text(0.5, 0.5, 'No year data available', ha='center', va='center')
                    axes[1, 0].set_title('Movies by Year (No Data)')
            except Exception as e:
                axes[1, 0].text(0.5, 0.5, f'Error: {str(e)}', ha='center', va='center')
                axes[1, 0].set_title('Movies by Year (Error)')
            
            # User activity
            try:
                user_activity = ratings_df['userId'].value_counts().head(20)
                if len(user_activity) > 0:
                    axes[1, 1].bar(range(len(user_activity)), user_activity.values, color='orange', alpha=0.7)
                    axes[1, 1].set_title('Top 20 Most Active Users')
                    axes[1, 1].set_xlabel('User Rank')
                    axes[1, 1].set_ylabel('Number of Ratings')
                else:
                    axes[1, 1].text(0.5, 0.5, 'No user data available', ha='center', va='center')
                    axes[1, 1].set_title('User Activity (No Data)')
            except Exception as e:
                axes[1, 1].text(0.5, 0.5, f'Error: {str(e)}', ha='center', va='center')
                axes[1, 1].set_title('User Activity (Error)')
            
            plt.tight_layout()
            plt.show()
            
    except Exception as e:
        with analytics_output:
            clear_output()
            print(f"❌ Error generating analytics: {str(e)}")
            print(f"📋 Debug info:")
            print(f"   - Movies DataFrame shape: {movies_df.shape if 'movies_df' in globals() else 'Not available'}")
            print(f"   - Ratings DataFrame shape: {ratings_df.shape if 'ratings_df' in globals() else 'Not available'}")
            print(f"   - Movies columns: {list(movies_df.columns) if 'movies_df' in globals() else 'Not available'}")
            print(f"   - Ratings columns: {list(ratings_df.columns) if 'ratings_df' in globals() else 'Not available'}")

analytics_button.on_click(on_analytics_button_click)

# Display analytics dashboard
analytics_box = widgets.VBox([
    widgets.HTML("<h3>📊 Data Analytics Dashboard</h3>"),
    widgets.HBox([analytics_button]),
    analytics_output
], layout=widgets.Layout(border='2px solid #6c757d', padding='10px', margin='10px'))

display(analytics_box)